# 벡터화로 특성 생성하기

유용한 모델을 훈련하게 위해 먼저 모델이 사용할 관련성이 있는 특성을 찾아야 합니다.

데이터를 탐색하여 유용한 패턴과 특성을 찾아 보겠습니다. 먼저 데이터를 벡터 공간에 임베딩하겠습니다.

In [5]:
from sklearn.model_selection import GroupShuffleSplit


def format_raw_df(df):
    """
    데이터를 정제하고 질문과 대답을 합칩니다.
    """
    df["PostTypeId"] = df["PostTypeId"].astype(int)
    df["Id"] = df["Id"].astype(int)
    df["AnswerCount"] = df["AnswerCount"].fillna(-1)
    df["AnswerCount"] = df["AnswerCount"].astype(int)
    df["OwnerUserId"].fillna(-1, inplace=True)
    df["OwnerUserId"] = df["OwnerUserId"].astype(int)
    df.set_index("Id", inplace=True, drop=False)

    df["is_question"] = df["PostTypeId"] == 1

    # 문서화된 것 이외의 PostTypeId를 필터링한다
    df = df[df["PostTypeId"].isin([1, 2])]

    # 질문과 대답을 연결합니다
    df = df.join(
        df[["Id", "Title", "body_text", "Score", "AcceptedAnswerId"]],
        on="ParentId",
        how="left",
        rsuffix="_question",
    )
    return df


def get_split_by_author(
        posts, author_id_column="OwnerUserId", test_size=0.3, random_state=40
):
    """
    훈련 세트와 테스트 세트로 나눕니다.
    작성자가 두 세트 중에 하나에만 등장하는 것을 보장합니다.
    :param posts: 모든 포스트와 레이블
    :param author_id_column: author_id가 들어 있는 열 이름
    :param test_size: 테스트 세트로 할당할 비율
    :param random_state: 랜덤 시드
    """
    splitter = GroupShuffleSplit(
        n_splits=1, test_size=test_size, random_state=random_state
    )
    splits = splitter.split(posts, groups=posts[author_id_column])
    train_idx, test_idx = next(splits)
    return posts.iloc[train_idx, :], posts.iloc[test_idx, :]


def get_normalized_series(df, col):
    """
    DataFrame 열을 정규화합니다.
    :param df: DataFrame
    :param col: 열 이름
    :return: Z-점수를 사용해 정규화된 시리즈 객체
    """
    return (df[col] - df[col].mean()) / df[col].std()


def add_v1_features(df):
    """
    입력 DataFrame에 첫 번째 특성을 추가합니다.
    :param df: 질문 DataFrame
    :return: 특성이 추가된 DataFrame
    """
    df["action_verb_full"] = (
        df["full_text"].str.contains("can", regex=False)
        | df["full_text"].str.contains("What", regex=False)
        | df["full_text"].str.contains("should", regex=False)
    )
    df["language_question"] = (
        df["full_text"].str.contains("punctuate", regex=False)
        | df["full_text"].str.contains("capitalize", regex=False)
        | df["full_text"].str.contains("abbreviate", regex=False)
    )
    df["question_mark_full"] = df["full_text"].str.contains("?", regex=False)
    df["text_len"] = df["full_text"].str.len()
    return df


def add_text_features_to_df(df):
    """
    DataFrame에 특성을 추가합니다.
    :param df: DataFrame
    :return: 특성이 추가된 DataFrame
    """
    df["full_text"] = df["Title"].str.cat(df["body_text"], sep=" ", na_rep="")
    df = add_v1_features(df.copy())

    return df

In [6]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


def plot_embeddings(embeddings, sent_labels):
    """
    문장 레이블에 따라 색을 입힌 임베딩 그래프 그리기
    :param embeddings: 2차원 임베딩
    :param sent_labels: 출력할 레이블
    """
    fig = plt.figure(figsize=(16, 10))
    color_map = {True: "#1f77b4", False: "#ff7f0e"}
    plt.scatter(
        embeddings[:, 0],
        embeddings[:, 1],
        c=[color_map[x] for x in sent_labels],
        s=40,
        alpha=0.4,
    )

    handles = [
        Rectangle((0, 0), 1, 1, color=c, ec="k") for c in ["#1f77b4", "#ff7f0e"]
    ]
    labels = ["answered", "unanswered"]
    plt.legend(handles, labels)

    plt.gca().set_aspect("equal", "box")
    plt.gca().set_xlabel("x")
    plt.gca().set_ylabel("y")

In [7]:
import pandas as pd
import spacy
import umap
import numpy as np
from io import BytesIO
from PIL import Image
import base64
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10, Category10
from pathlib import Path
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

# from ml_editor.data_processing import format_raw_df, get_split_by_author, get_normalized_series, add_text_features_to_df
# from ml_editor.data_visualization import plot_embeddings

df = pd.read_csv(Path('data/writers.csv'))
df = format_raw_df(df.copy())

train_author, test_author = get_split_by_author(df[df["is_question"]])

questions = train_author[train_author["is_question"]]
raw_text = questions["body_text"]
sent_labels = questions["AcceptedAnswerId"].notna()

# 라지 모델을 로드하고 불필요한 기능을 비활성화 합니다.
# 이렇게 하면 벡터화 처리 속도를 크게 높입니다.
# 모델에 관한 자세한 정보는 https://spacy.io/models/en#en_core_web_lg 를 참고하세요.
nlp = spacy.load('en_core_web_lg', disable=["parser", "tagger", "ner",
                                            "textcat", "lemmatizer"])

# 각 질문에 대한 벡터를 만듭니다
# 기본적으로 반환된 벡터는 문장에 있는 모든 벡터의 평균입니다.
# 자세한 정보는 https://spacy.io/usage/vectors-similarity 를 참고하세요.
spacy_emb = train_author[train_author["is_question"]]["body_text"].apply(lambda x: nlp(x).vector)
embeddings = np.vstack(spacy_emb)

umap_embedder = umap.UMAP()
umap_emb = umap_embedder.fit_transform(embeddings)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 인터랙티브 플롯

[보케(Bokeh)](https://docs.bokeh.org/en/latest/index.html)를 사용해 임베딩을 인터랙티브하게 살펴 보겠습니다.

왼쪽과 오른쪽으로 회전하고 관심 영역을 줌인 할 수 있습니다. 질문 위에 마우스를 올리면 연결된 텍스트와 레이블이 나타납니다.

In [8]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10, Category10

output_notebook()

def get_interactive_umap_embeddings_plot(umap_vectors, labels, text, legends, tooltip_label=None):
    if not tooltip_label:
        print("Using standard label")
        tooltip_label = labels
    w2v_df = pd.DataFrame(umap_vectors, columns=('x', 'y'))
    print(len(w2v_df))
    w2v_df['label'] = [str(x) for x in labels]
    w2v_df['tooltip_label'] = [str(x) for x in tooltip_label]
    w2v_df['text'] = list(text)
    w2v_df['legends'] = ["Answered" if el else "Unanswered" for el in list(legends)]
    datasource = ColumnDataSource(w2v_df)

    color_mapping = CategoricalColorMapper(factors=['True','False'], palette=['#1f77b4', '#ff7f0e'])

    TOOLTIPS = [
        ("text", "@text"),
        ('got_answer', '@tooltip_label')
    ]
    hover = HoverTool(tooltips=TOOLTIPS)
    hover.attachment ='right'

    plot_figure = figure(
        title='UMAP projection of questions',
        plot_width=900,
        plot_height=600,
        tools=('pan, wheel_zoom, reset', 'box_zoom', 'undo')
    )
    plot_figure.add_tools(hover)

    plot_figure.circle(
        'x',
        'y',
        source=datasource,
        color=dict(field='label', transform=color_mapping),
        legend='legends',
        line_alpha=0,
        fill_alpha=0.4,
        size=5
    )
    return plot_figure

plot_figure = get_interactive_umap_embeddings_plot(umap_emb, sent_labels, raw_text, legends=sent_labels)
show(plot_figure)

Loading BokehJS ...

Using standard label
5676


AttributeError: unexpected attribute 'plot_width' to figure, similar attributes are outer_width, width or min_width

데이터를 살펴 볼 때 마우스로 가리킨 특정 샘플 위치를 알 수 있도록 헬퍼 함수를 만들겠습니다. 아래 함수는 주어진 문자열을 담고 있는 샘플을 찾고 관련 특성을 출력합니다.

In [ ]:
# 검색을 위해서 빈 문자열로 만듭니다.
df["body_text_question"].fillna("", inplace=True)

def show_question_features_containing(text):
    return df[df["body_text_question"].str.contains(text)][["body_text", "CommentCount",
                                                             "body_text_question",
                                                       "Score_question", "AcceptedAnswerId_question"]]

# 비슷한 질문을 찾는 좋은 예
show_question_features_containing("I'm an amateur writer")

## 가능성있는 특성

임베딩과 관련된 데이터 행을 살펴 보니 질문의 타깃 클래스를 예측하는데 유용한 몇 개의 특성을 볼 수 있습니다. 그 중 몇 개는 다음과 같습니다:

- 질문 길이: 매우 짧은 질문은 대답을 받지 못하는 것 같습니다.
- 물음표 여부: 물음표가 없으면 답변을 받을 가능성이 낮아 보입니다.
- 명확한 질문에 관련된 어휘(동작 동사 등...): 대답을 받지 못한 질문에 빠져 있는 것 같습니다.

이외에 다른 것을 찾은 게 있나요? 그렇다면 마음껏 추가해도 좋습니다.

먼저 물음표와 동작 동사의 존재 여부를 특성으로 만들겠습니다.

In [ ]:
df["action_verb"] = (df["body_text"].str.contains("can", regex=False) | df["body_text"].str.contains("What", regex=False) | df["body_text"].str.contains("should", regex=False))
df["question_mark"] = df["body_text"].str.contains("?", regex=False)
df["text_len"] = df["body_text"].str.len()

각 특성이 `True`인 개수를 확인해 보죠.

In [ ]:
df["action_verb"].value_counts()

In [ ]:
df["question_mark"].value_counts()

텍스트 길이는 이미 측정했기 때문에 이를 정규화하는 것으로 간단히 전처리하겠습니다. 훈련 세트가 아니라 데이터프레임에 특성을 추가했기 때문에 훈련 세트를 다시 만듭니다.

In [ ]:
df["norm_text_len"]= get_normalized_series(df, "text_len")

In [ ]:
train_author, test_author = get_split_by_author(df[df["is_question"]])

이제 특성을 추가했으므로 새로운 임베딩을 만들기 위해 기존 벡터에 생성된 특성을 추가하고 임베딩 공간에 다시 시각화합니다.
특성이 많이 추가될수록 임베딩이 잘 구분되어야 합니다.

In [ ]:
vectorized_features = np.append(np.array(embeddings), train_author[train_author["is_question"]][["action_verb","question_mark",
                                                                            "norm_text_len"]], 1)

vectorized_features.shape

임베딩이 바뀌었으므로 그래프를 그리기 전에 UMAP 투영을 다시 계산합니다.

In [ ]:
umap_embedder = umap.UMAP()
umap_features = umap_embedder.fit_transform(vectorized_features)

In [ ]:
plot_embeddings(umap_features, sent_labels)

이 특성들이 데이터셋의 지형을 형성하는데 기여한 것으로 보입니다. 특히 대답을 받지 못한 질문의 대부분을 포함한 클러스터를 만든 것 같습니다. 이는 고무적이며 도움이 되는 특성이라는 신호입니다. 하지만 확신을 가지려면 모델에 사용해봐야 합니다.

데이터 세트를 다시 한 번 살펴보고 다른 특성을 도출할 수 있는지 살펴보겠습니다. 이렇게 하려면 아래 셀의 주석을 제거하세요. 노트북 로딩 속도를 높이기 위해 주석 처리한 상태로 둡니다. 자유롭게 `show_question_features_keeping` 함수를 사용해 가설을 검증해 보세요.

In [ ]:
# plot_figure = get_interactive_umap_embeddings_plot(umap_features, sent_labels, raw_text, legends=sent_labels)
# show(plot_figure)

이번 탐색에서 찾은 예측 성능을 가진 다른 특성은 질문이 영어에 관한 것인지 여부였습니다. 아래는 바로 캡처를 시도하는 기능입니다. 다음은 이런 질문을 잡아내는 특성입니다.

In [ ]:
df["language_question"] = (df["body_text"].str.contains("punctuate", regex=False) | df["body_text"].str.contains("capitalize", regex=False) | df["body_text"].str.contains("abbreviate", regex=False)).astype(int)

데이터를 다시 살펴 볼 때 질문의 제목에 관련성 있는 정보가 많다는 것을 알았습니다. 지금까지는 이를 무시했습니다. 다음은 많은 정보를 드러내는 제목을 가진 샘플입니다.

In [ ]:
df[df["body_text"].str.contains("Specifically, how to describe", regex=False)][["body_text", "Title"]]

제목과 본문을 연결한 다음 이를 다시 임베딩하겠습니다.

In [ ]:
df["full_text"] = df["Title"].str.cat(df["body_text"], sep=' ', na_rep='')

동일한 방식으로 전체 텍스트를 사용해 특성을 다시 만듭니다. 자세한 내용은 `ml_editor` 폴더를 참고하세요.

In [ ]:
df = add_text_features_to_df(df.loc[df["is_question"]].copy())

In [ ]:
train_author, test_author = get_split_by_author(df[df["is_question"]])
train_labels  = train_author["AcceptedAnswerId"].notna()

train_author["vectors"] = train_author["full_text"].apply(lambda x: nlp(x).vector)

In [ ]:
vectorized_features = np.append(
        np.vstack(train_author["vectors"]),
        train_author[
            [
                "action_verb_full",
                "question_mark_full",
                "norm_text_len",
                "language_question",
            ]
        ],
        1,
    )

임베딩을 다시 시각화해 보겠습니다.

In [ ]:
umap_embedder = umap.UMAP()
umap_features = umap_embedder.fit_transform(vectorized_features)

In [ ]:
plot_embeddings(umap_features, train_labels)

위 그림에서 대부분 대답을 얻지 못한 질문을 담고 있는 몇 개의 클러스터가 상단에 고립되어 있는 것을 볼 수 있습니다. 추가된 특성이 클래스를 분할하는데 도움이 되기 때문에 모델의 작업을 쉽게 만들 것입니다.

데이터를 더 탐색하고 싶다면 아래 코드의 주석을 제거하고 실행하세요.

In [ ]:
plot_figure = get_interactive_umap_embeddings_plot(umap_features, sent_labels, raw_text, legends=sent_labels)
show(plot_figure)

예측 성능을 기대하는 후보 특성으로 초기 데이터셋을 만들었습니다. 이제 이 데이터셋에 모델을 훈련하고 성능을 조사하여 반복할 수 있습니다!